# Vertex AI SDK for Python: Vertex AI Forecasting Model Training Results

This notebook provides access to example output data saved to BigQuery from a batch prediction made by a Vertex AI AutoML Forecasting Model. This allows you to see,and interact with, sample result data without waiting for the model to complete training in the lab. 

The first three cells of this lab define the environment variables that allow you to run the data visualization step from the main lab notebook against a pre-prepared example BigQuery dataset called **iowa_liquor_sales_predictions_example**. 

To use this notebook run each step, or cell, in seuqence and see its results. To run a cell, use Shift+Enter. The notebooks will automatically display the return value of the last line in each cell. 

## Import System Modules

In [ ]:
import sys
import os
from datetime import datetime

from google.cloud import bigquery

## Define Lab Environment variables

In [ ]:
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT_ID = shell_output[0]
print("Project ID: ", PROJECT_ID)
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_NAME = "gs://"+PROJECT_ID
REGION = "us-central1"  # Change this if you need to use a different region for Vertex AI

In [ ]:
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

time_column = "date"
time_series_identifier_column="store_name"
target_column="sale_dollars"

batch_predict_bq_input_uri = "bq://bigquery-public-data.iowa_liquor_sales_forecasting.2021_sales_predict"
batch_predict_bq_output_dataset_name = "iowa_liquor_sales_predictions_example"
batch_predict_bq_output_dataset_path = "{}.{}".format(PROJECT_ID, batch_predict_bq_output_dataset_name)
batch_predict_bq_output_uri_prefix = "bq://{}.{}".format(PROJECT_ID, batch_predict_bq_output_dataset_name)
# Must be the same region as batch_predict_bq_input_uri
client = bigquery.Client()

# Visualize the Example Batch Forecasts
Follow the this link to visualize the example batch prediction forecasts in [Data Studio](https://support.google.com/datastudio/answer/6283323?hl=en).

In [ ]:
import urllib

tables = client.list_tables(batch_predict_bq_output_dataset_path)

prediction_table_id = ""
for table in tables:
  if table.table_id.startswith(
      "predictions_") and table.table_id > prediction_table_id:
    prediction_table_id = table.table_id
batch_predict_bq_output_uri = "{}.{}".format(
    batch_predict_bq_output_dataset_path, prediction_table_id)


def _sanitize_bq_uri(bq_uri):
  if bq_uri.startswith("bq://"):
    bq_uri = bq_uri[5:]
  return bq_uri.replace(":", ".")


def get_data_studio_link(batch_prediction_bq_input_uri,
                         batch_prediction_bq_output_uri, time_column,
                         time_series_identifier_column, target_column):
  batch_prediction_bq_input_uri = _sanitize_bq_uri(
      batch_prediction_bq_input_uri)
  batch_prediction_bq_output_uri = _sanitize_bq_uri(
      batch_prediction_bq_output_uri)
  base_url = "https://datastudio.google.com/c/u/0/reporting"
  query = "SELECT \\n" \
  " CAST(input.{} as DATETIME) timestamp_col,\\n" \
  " CAST(input.{} as STRING) time_series_identifier_col,\\n" \
  " CAST(input.{} as NUMERIC) historical_values,\\n" \
  " CAST(predicted_{}.value as NUMERIC) predicted_values,\\n" \
  " * \\n" \
  "FROM `{}` input\\n" \
  "LEFT JOIN `{}` output\\n" \
  "ON\\n" \
  "CAST(input.{} as DATETIME) = CAST(output.{} as DATETIME)\\n" \
  "AND CAST(input.{} as STRING) = CAST(output.{} as STRING)"
  query = query.format(time_column, time_series_identifier_column,
                       target_column, target_column,
                       batch_prediction_bq_input_uri,
                       batch_prediction_bq_output_uri, time_column, time_column,
                       time_series_identifier_column,
                       time_series_identifier_column)
  params = {
      "templateId": "067f70d2-8cd6-4a4c-a099-292acd1053e8",
      "ds0.connector": "BIG_QUERY",
      "ds0.projectId": PROJECT_ID,
      "ds0.billingProjectId": PROJECT_ID,
      "ds0.type": "CUSTOM_QUERY",
      "ds0.sql": query
  }
  params_str_parts = []
  for k, v in params.items():
    params_str_parts.append("\"{}\":\"{}\"".format(k, v))
  params_str = "".join(["{", ",".join(params_str_parts), "}"])
  return "{}?{}".format(base_url,
                        urllib.parse.urlencode({"params": params_str}))


print(
    get_data_studio_link(batch_predict_bq_input_uri,
                         batch_predict_bq_output_uri, time_column,
                         time_series_identifier_column, target_column))

# Return to the main notebook
When you are finished exploring the data you can return to the main notebook to wait for the model to complete training if you wish to interact with the model directly and perform your own predictions. 

In [ ]:
# bq --location=us mk --dataset $PROJECT_ID:iowa_liquor_sales_predictions_example
# bq --location=us load --source_format=AVRO \
#    iowa_liquor_sales_predictions_example.errors_example \
#    gs://cloud-training/CBL458/setup/batch_prediction_error_example.avro
# bq --location=us load --source_format=AVRO \
#    iowa_liquor_sales_predictions_example.predictions_example \
#    gs://cloud-training/CBL458/setup/batch_prediction_example.avro